In [1]:
from collections import defaultdict
from collections import Counter
import json
import math
import string
import time
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from IPython.core.display import HTML
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
import re

In [2]:
with open("finalData.json", "r") as f:
    data = json.load(f)

In [3]:
#code from a5
def build_vectorizer():
    """Returns a TfidfVectorizer object with certain preprocessing properties.
    
    Params: {max_n_terms: Integer,
             max_prop_docs: Float,
             min_n_docs: Integer}
    Returns: TfidfVectorizer
    """
    return TfidfVectorizer(stop_words = 'english')

In [4]:
#code from class demo
word_splitter = re.compile(r"""
    (\w+)
    """, re.VERBOSE)

def getwords(sent):
    return [w.lower() 
            for w in word_splitter.findall(sent)]

In [ ]:
tfidf_vec = build_vectorizer()
stemmer=PorterStemmer()
reviews = []
for city, city_dic in data.items():
    for restaurant, restaurant_dic in city_dic.items():
        for review in restaurant_dic['reviews']:
            all_words = getwords(review['text'])
            stem_text = [stemmer.stem(t.lower()) for t in all_words]
            reviews.append(" ".join(stem_text))

In [ ]:
print(len(reviews))

In [ ]:
tfidf_mat = tfidf_vec.fit_transform(reviews).toarray()

In [ ]:
print(tfidf_mat[0])

In [ ]:
print(tfidf_vec.get_feature_names())

In [ ]:
print(len(tfidf_vec.get_feature_names()))

In [ ]:
#code from a5
def build_movie_sims_cos(start, end, cos_sim, input_doc_mat, num_movies, norms):
    """Returns a matrix of size num_movies x num_movies where for (i,j), entry [i,j]
       should be the cosine similarity between the movie with index i and the movie with index j
        
    Note: All movies are trivially perfectly similar to themselves, so the diagonals of the output matrix should be 1.
    
    Params: {num_movies: Integer,
             input_doc_mat: Numpy Array,
             movie_index_to_name: Dict,
             movie_name_to_index: Dict,
             input_get_sim_method: Function}
    Returns: Numpy Array 
    """
    #cos_sim = np.zeros((num_movies, num_movies))
    for i in range(start, end):
        if i >= num_movies:
            break
        for j in range(start, end):
            if j >= num_movies:
                break
            cos_sim[i][j] = np.dot(input_doc_mat[i], input_doc_mat[j])/(norms[i] * norms[j])
    return cos_sim

In [ ]:
#sims_cos = build_movie_sims_cos(len(reviews), tfidf_mat)

In [ ]:
cos_sim = np.zeros((len(reviews), len(reviews)))

In [ ]:
start = 0
end = 5000
norms = np.linalg.norm(tfidf_mat, axis=1)
last_round = False
finished = False
while not finished:
    print("start of loop")
    cos_sim = build_sim_movies(start, end, cos_sim, tfidf_mat, len(reviews), norms)
    start += 5001
    end += 5001
    if end > len(reviews) - 1:
        end = len(reviews) - 1
        if last_round:
            finished = True
        last_round = True

